In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
df = pd.read_csv('US Superstore.csv', encoding = 'latin-1')

In [3]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format = 'mixed')

In [4]:
grouped = df.groupby(['Customer ID', 'Segment', 'Sub-Category', 'Category']).agg(
    No_of_Orders = ('Order ID', 'nunique'),
    Total_Quantity = ('Quantity', 'sum'),
).reset_index()

In [5]:
orderDates = df.groupby(['Customer ID', 'Sub-Category'])['Order Date'].apply(list).reset_index()

In [6]:
orderDates

,Customer ID,Sub-Category,Order Date
0,AA-10315,Accessories,[2016-03-03 00:00:00]
1,AA-10315,Appliances,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]"
2,AA-10315,Binders,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]"
3,AA-10315,Fasteners,[2016-03-03 00:00:00]
4,AA-10315,Furnishings,[2014-09-15 00:00:00]
...,...,...,...
5997,ZD-21925,Art,[2014-08-27 00:00:00]
5998,ZD-21925,Binders,[2014-08-27 00:00:00]
5999,ZD-21925,Furnishings,"[2016-05-05 00:00:00, 2016-03-04 00:00:00]"
6000,ZD-21925,Paper,[2016-08-07 00:00:00]


In [7]:
mergedDates = pd.merge(grouped, orderDates, on = ['Customer ID', 'Sub-Category'], how = 'left')

In [8]:
mergedDates

,Customer ID,Segment,Sub-Category,Category,No_of_Orders,Total_Quantity,Order Date
0,AA-10315,Consumer,Accessories,Technology,1,7,[2016-03-03 00:00:00]
1,AA-10315,Consumer,Appliances,Office Supplies,2,5,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]"
2,AA-10315,Consumer,Binders,Office Supplies,2,4,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]"
3,AA-10315,Consumer,Fasteners,Office Supplies,1,1,[2016-03-03 00:00:00]
4,AA-10315,Consumer,Furnishings,Furniture,1,2,[2014-09-15 00:00:00]
...,...,...,...,...,...,...,...
5997,ZD-21925,Consumer,Art,Office Supplies,1,2,[2014-08-27 00:00:00]
5998,ZD-21925,Consumer,Binders,Office Supplies,1,4,[2014-08-27 00:00:00]
5999,ZD-21925,Consumer,Furnishings,Furniture,2,9,"[2016-05-05 00:00:00, 2016-03-04 00:00:00]"
6000,ZD-21925,Consumer,Paper,Office Supplies,1,2,[2016-08-07 00:00:00]


In [9]:
def calculateGaps(orderDates):
    uniqueDates = sorted(set(orderDates))

    if len(uniqueDates) > 1:
        gaps = [(uniqueDates[i] - uniqueDates[i-1]).days for i in range(1, len(uniqueDates))]
        minGap = min(gaps)
        maxGap = max(gaps)
        avgGap = sum(gaps) / len(gaps)
    else:
        minGap = maxGap = avgGap = 0
    return pd.Series([minGap, maxGap, avgGap], index = ['Minimum Gap', 'Maximum Gap', 'Average Gap'])

mergedDates[['Minimum Gaps', 'Maximum Gaps', 'Average Gaps']] = mergedDates['Order Date'].apply(calculateGaps)

In [10]:
firstDate = df.groupby('Customer ID')['Order Date'].min().reset_index()

In [11]:
firstDates = firstDate.rename(columns = {
    'Order Date' : 'First Date of Purchase'})

In [12]:
lastDate = df.groupby('Customer ID')['Order Date'].max().reset_index()

In [13]:
lastDates = lastDate.rename(columns = {
    'Order Date' : 'Last Date of Purchase'})

In [14]:
newMerged = pd.merge(mergedDates, firstDates, on = 'Customer ID', how = 'right')

In [15]:
newMerged = pd.merge(newMerged, lastDates, on = 'Customer ID', how = 'right')

In [16]:
newMerged

,Customer ID,Segment,Sub-Category,Category,No_of_Orders,Total_Quantity,Order Date,Minimum Gaps,Maximum Gaps,Average Gaps,First Date of Purchase,Last Date of Purchase
0,AA-10315,Consumer,Accessories,Technology,1,7,[2016-03-03 00:00:00],0.0,0.0,0.0,2014-03-31,2017-06-29
1,AA-10315,Consumer,Appliances,Office Supplies,2,5,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]",1186.0,1186.0,1186.0,2014-03-31,2017-06-29
2,AA-10315,Consumer,Binders,Office Supplies,2,4,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]",1186.0,1186.0,1186.0,2014-03-31,2017-06-29
3,AA-10315,Consumer,Fasteners,Office Supplies,1,1,[2016-03-03 00:00:00],0.0,0.0,0.0,2014-03-31,2017-06-29
4,AA-10315,Consumer,Furnishings,Furniture,1,2,[2014-09-15 00:00:00],0.0,0.0,0.0,2014-03-31,2017-06-29
...,...,...,...,...,...,...,...,...,...,...,...,...
5997,ZD-21925,Consumer,Art,Office Supplies,1,2,[2014-08-27 00:00:00],0.0,0.0,0.0,2014-08-27,2017-11-06
5998,ZD-21925,Consumer,Binders,Office Supplies,1,4,[2014-08-27 00:00:00],0.0,0.0,0.0,2014-08-27,2017-11-06
5999,ZD-21925,Consumer,Furnishings,Furniture,2,9,"[2016-05-05 00:00:00, 2016-03-04 00:00:00]",62.0,62.0,62.0,2014-08-27,2017-11-06
6000,ZD-21925,Consumer,Paper,Office Supplies,1,2,[2016-08-07 00:00:00],0.0,0.0,0.0,2014-08-27,2017-11-06


In [17]:
lastDate = '2017-12-31'
lastDate = pd.to_datetime(lastDate)

In [18]:
newMerged['Days Since Last Purchase'] = (lastDate - newMerged['Last Date of Purchase']).dt.days

In [19]:
lconsumerCustomers = newMerged[newMerged['Segment'] == 'Consumer']
lcorporateCustomers = newMerged[newMerged['Segment'] == 'Corporate']
lhomeofficeCustomers = newMerged[newMerged['Segment'] == 'Home Office']

In [20]:
consumerCount = 164
corporateCount = 143
homeofficeCount = 126

In [21]:
def lconflagCustomer(row):
    if row['Average Gaps'] == 0:
        if row['Days Since Last Purchase'] < 365:
            return 'Yes'
        else:
            return 'No'
    elif row['Average Gaps'] > consumerCount or row['Average Gaps'] == consumerCount:
        if row['Days Since Last Purchase'] < 365: 
            return 'Yes'
        else:
            return 'No'
    elif row['Average Gaps'] < consumerCount:
        return 'No'

lconsumerCustomers['Flagged 1'] = lconsumerCustomers.apply(lconflagCustomer, axis = 1)
lconsumerCustomers

C:\Users\shara\AppData\Local\Temp\ipykernel_33488\3607745899.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lconsumerCustomers['Flagged 1'] = lconsumerCustomers.apply(lconflagCustomer, axis = 1)


,Customer ID,Segment,Sub-Category,Category,No_of_Orders,Total_Quantity,Order Date,Minimum Gaps,Maximum Gaps,Average Gaps,First Date of Purchase,Last Date of Purchase,Days Since Last Purchase,Flagged 1
0,AA-10315,Consumer,Accessories,Technology,1,7,[2016-03-03 00:00:00],0.0,0.0,0.0,2014-03-31,2017-06-29,185,Yes
1,AA-10315,Consumer,Appliances,Office Supplies,2,5,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]",1186.0,1186.0,1186.0,2014-03-31,2017-06-29,185,Yes
2,AA-10315,Consumer,Binders,Office Supplies,2,4,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]",1186.0,1186.0,1186.0,2014-03-31,2017-06-29,185,Yes
3,AA-10315,Consumer,Fasteners,Office Supplies,1,1,[2016-03-03 00:00:00],0.0,0.0,0.0,2014-03-31,2017-06-29,185,Yes
4,AA-10315,Consumer,Furnishings,Furniture,1,2,[2014-09-15 00:00:00],0.0,0.0,0.0,2014-03-31,2017-06-29,185,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,ZD-21925,Consumer,Art,Office Supplies,1,2,[2014-08-27 00:00:00],0.0,0.0,0.0,2014-08-27,2017-11-06,55,Yes
5998,ZD-21925,Consumer,Binders,Office Supplies,1,4,[2014-08-27 00:00:00],0.0,0.0,0.0,2014-08-27,2017-11-06,55,Yes
5999,ZD-21925,Consumer,Furnishings,Furniture,2,9,"[2016-05-05 00:00:00, 2016-03-04 00:00:00]",62.0,62.0,62.0,2014-08-27,2017-11-06,55,No
6000,ZD-21925,Consumer,Paper,Office Supplies,1,2,[2016-08-07 00:00:00],0.0,0.0,0.0,2014-08-27,2017-11-06,55,Yes


In [22]:
def lcorflagCustomer(row):
    if row['Average Gaps'] == 0:
        if row['Days Since Last Purchase'] < 365:
            return 'Yes'
        else:
            return 'No'
    elif row['Average Gaps'] > corporateCount or row['Average Gaps'] == corporateCount:
        if row['Days Since Last Purchase'] < 365: 
            return 'Yes'
        else:
            return 'No'
    elif row['Average Gaps'] < corporateCount:
        return 'No'

lcorporateCustomers['Flagged 1'] = lcorporateCustomers.apply(lcorflagCustomer, axis = 1)
lcorporateCustomers

C:\Users\shara\AppData\Local\Temp\ipykernel_33488\1251782408.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lcorporateCustomers['Flagged 1'] = lcorporateCustomers.apply(lcorflagCustomer, axis = 1)


,Customer ID,Segment,Sub-Category,Category,No_of_Orders,Total_Quantity,Order Date,Minimum Gaps,Maximum Gaps,Average Gaps,First Date of Purchase,Last Date of Purchase,Days Since Last Purchase,Flagged 1
77,AB-10600,Corporate,Accessories,Technology,1,6,[2017-10-11 00:00:00],0.0,0.0,0.000000,2014-11-18,2017-10-11,81,Yes
78,AB-10600,Corporate,Binders,Office Supplies,2,19,"[2017-10-11 00:00:00, 2014-11-18 00:00:00]",1058.0,1058.0,1058.000000,2014-11-18,2017-10-11,81,Yes
79,AB-10600,Corporate,Chairs,Furniture,1,4,[2017-10-11 00:00:00],0.0,0.0,0.000000,2014-11-18,2017-10-11,81,Yes
80,AB-10600,Corporate,Furnishings,Furniture,1,2,[2015-11-30 00:00:00],0.0,0.0,0.000000,2014-11-18,2017-10-11,81,Yes
81,AB-10600,Corporate,Labels,Office Supplies,1,3,[2015-02-18 00:00:00],0.0,0.0,0.000000,2014-11-18,2017-10-11,81,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5979,YS-21880,Corporate,Chairs,Furniture,1,6,[2017-04-13 00:00:00],0.0,0.0,0.000000,2015-07-25,2017-12-21,10,Yes
5980,YS-21880,Corporate,Envelopes,Office Supplies,1,9,[2015-07-25 00:00:00],0.0,0.0,0.000000,2015-07-25,2017-12-21,10,Yes
5981,YS-21880,Corporate,Furnishings,Furniture,1,3,[2017-08-18 00:00:00],0.0,0.0,0.000000,2015-07-25,2017-12-21,10,Yes
5982,YS-21880,Corporate,Storage,Office Supplies,4,19,"[2017-12-21 00:00:00, 2015-12-01 00:00:00, 201...",111.0,626.0,287.333333,2015-07-25,2017-12-21,10,Yes


In [23]:
def lhomeofficeflagCustomer(row):
    if row['Average Gaps'] == 0:
        if row['Days Since Last Purchase'] < 365:
            return 'Yes'
        else:
            return 'No'
    elif row['Average Gaps'] > homeofficeCount or row['Average Gaps'] == homeofficeCount:
        if row['Days Since Last Purchase'] < 365: 
            return 'Yes'
        else:
            return 'No'
    elif row['Average Gaps'] < homeofficeCount:
        return 'No'
        
lhomeofficeCustomers['Flagged 1'] = lhomeofficeCustomers.apply(lhomeofficeflagCustomer, axis = 1)
lhomeofficeCustomers

C:\Users\shara\AppData\Local\Temp\ipykernel_33488\2842758072.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lhomeofficeCustomers['Flagged 1'] = lhomeofficeCustomers.apply(lhomeofficeflagCustomer, axis = 1)


,Customer ID,Segment,Sub-Category,Category,No_of_Orders,Total_Quantity,Order Date,Minimum Gaps,Maximum Gaps,Average Gaps,First Date of Purchase,Last Date of Purchase,Days Since Last Purchase,Flagged 1
37,AB-10060,Home Office,Accessories,Technology,3,5,"[2016-01-09 00:00:00, 2017-07-05 00:00:00, 201...",24.0,519.0,271.5,2015-09-18,2017-09-16,106,Yes
38,AB-10060,Home Office,Appliances,Office Supplies,1,2,[2016-03-13 00:00:00],0.0,0.0,0.0,2015-09-18,2017-09-16,106,Yes
39,AB-10060,Home Office,Binders,Office Supplies,2,9,"[2016-01-09 00:00:00, 2016-01-09 00:00:00, 201...",64.0,64.0,64.0,2015-09-18,2017-09-16,106,No
40,AB-10060,Home Office,Chairs,Furniture,1,2,[2017-06-11 00:00:00],0.0,0.0,0.0,2015-09-18,2017-09-16,106,Yes
41,AB-10060,Home Office,Furnishings,Furniture,1,2,[2016-03-13 00:00:00],0.0,0.0,0.0,2015-09-18,2017-09-16,106,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5930,VT-21700,Home Office,Bookcases,Furniture,1,5,[2015-05-04 00:00:00],0.0,0.0,0.0,2014-06-06,2015-05-04,972,No
5931,VT-21700,Home Office,Furnishings,Furniture,1,5,"[2015-05-04 00:00:00, 2015-05-04 00:00:00]",0.0,0.0,0.0,2014-06-06,2015-05-04,972,No
5932,VT-21700,Home Office,Labels,Office Supplies,1,10,[2014-06-06 00:00:00],0.0,0.0,0.0,2014-06-06,2015-05-04,972,No
5933,VT-21700,Home Office,Phones,Technology,1,7,"[2015-05-04 00:00:00, 2015-05-04 00:00:00]",0.0,0.0,0.0,2014-06-06,2015-05-04,972,No


In [24]:
def flag2(row):
    if row['Average Gaps'] > consumerCount:
        return 'Yes'
    else:
        return 'No'


lconsumerCustomers['Flagged 2'] = lconsumerCustomers.apply(flag2, axis = 1)
lconsumerCustomers

C:\Users\shara\AppData\Local\Temp\ipykernel_33488\2133276985.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lconsumerCustomers['Flagged 2'] = lconsumerCustomers.apply(flag2, axis = 1)


,Customer ID,Segment,Sub-Category,Category,No_of_Orders,Total_Quantity,Order Date,Minimum Gaps,Maximum Gaps,Average Gaps,First Date of Purchase,Last Date of Purchase,Days Since Last Purchase,Flagged 1,Flagged 2
0,AA-10315,Consumer,Accessories,Technology,1,7,[2016-03-03 00:00:00],0.0,0.0,0.0,2014-03-31,2017-06-29,185,Yes,No
1,AA-10315,Consumer,Appliances,Office Supplies,2,5,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]",1186.0,1186.0,1186.0,2014-03-31,2017-06-29,185,Yes,Yes
2,AA-10315,Consumer,Binders,Office Supplies,2,4,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]",1186.0,1186.0,1186.0,2014-03-31,2017-06-29,185,Yes,Yes
3,AA-10315,Consumer,Fasteners,Office Supplies,1,1,[2016-03-03 00:00:00],0.0,0.0,0.0,2014-03-31,2017-06-29,185,Yes,No
4,AA-10315,Consumer,Furnishings,Furniture,1,2,[2014-09-15 00:00:00],0.0,0.0,0.0,2014-03-31,2017-06-29,185,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,ZD-21925,Consumer,Art,Office Supplies,1,2,[2014-08-27 00:00:00],0.0,0.0,0.0,2014-08-27,2017-11-06,55,Yes,No
5998,ZD-21925,Consumer,Binders,Office Supplies,1,4,[2014-08-27 00:00:00],0.0,0.0,0.0,2014-08-27,2017-11-06,55,Yes,No
5999,ZD-21925,Consumer,Furnishings,Furniture,2,9,"[2016-05-05 00:00:00, 2016-03-04 00:00:00]",62.0,62.0,62.0,2014-08-27,2017-11-06,55,No,No
6000,ZD-21925,Consumer,Paper,Office Supplies,1,2,[2016-08-07 00:00:00],0.0,0.0,0.0,2014-08-27,2017-11-06,55,Yes,No


In [25]:
def flag2(row):
    if row['Average Gaps'] > corporateCount:
        return 'Yes'
    else:
        return 'No'


lcorporateCustomers['Flagged 2'] = lcorporateCustomers.apply(flag2, axis = 1)
lcorporateCustomers

C:\Users\shara\AppData\Local\Temp\ipykernel_33488\3887288817.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lcorporateCustomers['Flagged 2'] = lcorporateCustomers.apply(flag2, axis = 1)


,Customer ID,Segment,Sub-Category,Category,No_of_Orders,Total_Quantity,Order Date,Minimum Gaps,Maximum Gaps,Average Gaps,First Date of Purchase,Last Date of Purchase,Days Since Last Purchase,Flagged 1,Flagged 2
77,AB-10600,Corporate,Accessories,Technology,1,6,[2017-10-11 00:00:00],0.0,0.0,0.000000,2014-11-18,2017-10-11,81,Yes,No
78,AB-10600,Corporate,Binders,Office Supplies,2,19,"[2017-10-11 00:00:00, 2014-11-18 00:00:00]",1058.0,1058.0,1058.000000,2014-11-18,2017-10-11,81,Yes,Yes
79,AB-10600,Corporate,Chairs,Furniture,1,4,[2017-10-11 00:00:00],0.0,0.0,0.000000,2014-11-18,2017-10-11,81,Yes,No
80,AB-10600,Corporate,Furnishings,Furniture,1,2,[2015-11-30 00:00:00],0.0,0.0,0.000000,2014-11-18,2017-10-11,81,Yes,No
81,AB-10600,Corporate,Labels,Office Supplies,1,3,[2015-02-18 00:00:00],0.0,0.0,0.000000,2014-11-18,2017-10-11,81,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5979,YS-21880,Corporate,Chairs,Furniture,1,6,[2017-04-13 00:00:00],0.0,0.0,0.000000,2015-07-25,2017-12-21,10,Yes,No
5980,YS-21880,Corporate,Envelopes,Office Supplies,1,9,[2015-07-25 00:00:00],0.0,0.0,0.000000,2015-07-25,2017-12-21,10,Yes,No
5981,YS-21880,Corporate,Furnishings,Furniture,1,3,[2017-08-18 00:00:00],0.0,0.0,0.000000,2015-07-25,2017-12-21,10,Yes,No
5982,YS-21880,Corporate,Storage,Office Supplies,4,19,"[2017-12-21 00:00:00, 2015-12-01 00:00:00, 201...",111.0,626.0,287.333333,2015-07-25,2017-12-21,10,Yes,Yes


In [26]:
def flag2(row):
    if row['Average Gaps'] > homeofficeCount:
        return 'Yes'
    else:
        return 'No'


lhomeofficeCustomers['Flagged 2'] = lhomeofficeCustomers.apply(flag2, axis = 1)
lhomeofficeCustomers

C:\Users\shara\AppData\Local\Temp\ipykernel_33488\500321446.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lhomeofficeCustomers['Flagged 2'] = lhomeofficeCustomers.apply(flag2, axis = 1)


,Customer ID,Segment,Sub-Category,Category,No_of_Orders,Total_Quantity,Order Date,Minimum Gaps,Maximum Gaps,Average Gaps,First Date of Purchase,Last Date of Purchase,Days Since Last Purchase,Flagged 1,Flagged 2
37,AB-10060,Home Office,Accessories,Technology,3,5,"[2016-01-09 00:00:00, 2017-07-05 00:00:00, 201...",24.0,519.0,271.5,2015-09-18,2017-09-16,106,Yes,Yes
38,AB-10060,Home Office,Appliances,Office Supplies,1,2,[2016-03-13 00:00:00],0.0,0.0,0.0,2015-09-18,2017-09-16,106,Yes,No
39,AB-10060,Home Office,Binders,Office Supplies,2,9,"[2016-01-09 00:00:00, 2016-01-09 00:00:00, 201...",64.0,64.0,64.0,2015-09-18,2017-09-16,106,No,No
40,AB-10060,Home Office,Chairs,Furniture,1,2,[2017-06-11 00:00:00],0.0,0.0,0.0,2015-09-18,2017-09-16,106,Yes,No
41,AB-10060,Home Office,Furnishings,Furniture,1,2,[2016-03-13 00:00:00],0.0,0.0,0.0,2015-09-18,2017-09-16,106,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5930,VT-21700,Home Office,Bookcases,Furniture,1,5,[2015-05-04 00:00:00],0.0,0.0,0.0,2014-06-06,2015-05-04,972,No,No
5931,VT-21700,Home Office,Furnishings,Furniture,1,5,"[2015-05-04 00:00:00, 2015-05-04 00:00:00]",0.0,0.0,0.0,2014-06-06,2015-05-04,972,No,No
5932,VT-21700,Home Office,Labels,Office Supplies,1,10,[2014-06-06 00:00:00],0.0,0.0,0.0,2014-06-06,2015-05-04,972,No,No
5933,VT-21700,Home Office,Phones,Technology,1,7,"[2015-05-04 00:00:00, 2015-05-04 00:00:00]",0.0,0.0,0.0,2014-06-06,2015-05-04,972,No,No


In [27]:
rfm = df.groupby('Customer ID').agg(
    Recency=('Order Date', lambda day: (lastDate - day.max()).days),
    Frequency=('Order ID', 'nunique'),
    Monetary=('Sales', 'sum')
).reset_index()
rfm

,Customer ID,Recency,Frequency,Monetary
0,AA-10315,185,5,5563.560
1,AA-10375,49,9,1056.390
2,AA-10480,260,4,1790.512
3,AA-10645,234,6,5086.935
4,AB-10015,446,3,886.156
...,...,...,...,...
788,XP-21865,44,11,2374.658
789,YC-21895,5,5,5454.350
790,YS-21880,10,8,6720.444
791,ZC-21910,55,13,8025.707


In [28]:
rfm['Recency Score'] = pd.qcut(rfm['Recency'], 5, labels = [5,4,3,2,1])
rfm["Frequency Score"] = pd.qcut(rfm["Frequency"],5,labels=[1,2,3,4,5])
rfm["Monetary Score"] = pd.qcut(rfm["Monetary"],5,labels=[1,2,3,4,5])
rfm['RFM Score'] = rfm['Recency Score'].astype(str) + rfm['Frequency Score'].astype(str) + rfm['Monetary Score'].astype(str)

## Cross-over Points for Recency:

In [29]:
recencyQuantiles = rfm['Recency'].quantile([0.2, 0.4, 0.6, 0.8, 1])
recencyQuantiles.reset_index()

,index,Recency
0,0.2,32.4
1,0.4,69.0
2,0.6,127.0
3,0.8,268.4
4,1.0,1166.0


## Cross-over Points for Frequency:

In [30]:
frequencyQuantiles = rfm['Frequency'].quantile([0.2, 0.4, 0.6, 0.8, 1])
frequencyQuantiles.reset_index()

,index,Frequency
0,0.2,4.0
1,0.4,5.0
2,0.6,7.0
3,0.8,8.6
4,1.0,17.0


## Cross-over Points for Monetary:

In [31]:
monetaryQuantiles = rfm['Monetary'].quantile([0.2, 0.4, 0.6, 0.8, 1])
monetaryQuantiles.reset_index()

,index,Monetary
0,0.2,961.5406
1,0.4,1739.4324
2,0.6,2750.7480
3,0.8,4282.6344
4,1.0,25043.0500


In [32]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating/At Risk',
    r'[1-2][3-5]': 'Can\'t Lose',
    r'3[1-3]': 'Need Attention',
    r'[4-5][1-3]': 'Promising/Potential Loyalists',
    r'[4-5][4-5]': 'Champions/Loyal Customers',
    r'51': 'New Customers',
    r'3[4-5]': 'Frequent but not Recent',
}

In [33]:
rfm['RFM Segment'] = rfm['Recency Score'].astype(str) + rfm['Frequency Score'].astype(str)
rfm['RFM Segment'] = rfm['RFM Segment'].replace(seg_map, regex=True)
rfm

,Customer ID,Recency,Frequency,Monetary,Recency Score,Frequency Score,Monetary Score,RFM Score,RFM Segment
0,AA-10315,185,5,5563.560,2,2,5,225,Hibernating/At Risk
1,AA-10375,49,9,1056.390,4,5,2,452,Champions/Loyal Customers
2,AA-10480,260,4,1790.512,2,1,3,213,Hibernating/At Risk
3,AA-10645,234,6,5086.935,2,3,5,235,Can't Lose
4,AB-10015,446,3,886.156,1,1,1,111,Hibernating/At Risk
...,...,...,...,...,...,...,...,...,...
788,XP-21865,44,11,2374.658,4,5,3,453,Champions/Loyal Customers
789,YC-21895,5,5,5454.350,5,2,5,525,Promising/Potential Loyalists
790,YS-21880,10,8,6720.444,5,4,5,545,Champions/Loyal Customers
791,ZC-21910,55,13,8025.707,4,5,5,455,Champions/Loyal Customers


In [34]:
customerSegment = df.groupby('Customer ID')['Segment'].unique().reset_index()

In [35]:
customerSegment

,Customer ID,Segment
0,AA-10315,[Consumer]
1,AA-10375,[Consumer]
2,AA-10480,[Consumer]
3,AA-10645,[Consumer]
4,AB-10015,[Consumer]
...,...,...
788,XP-21865,[Consumer]
789,YC-21895,[Corporate]
790,YS-21880,[Corporate]
791,ZC-21910,[Consumer]


In [36]:
rfm = pd.merge(customerSegment, rfm, on = 'Customer ID', how = 'left')

In [37]:
rfm

,Customer ID,Segment,Recency,Frequency,Monetary,Recency Score,Frequency Score,Monetary Score,RFM Score,RFM Segment
0,AA-10315,[Consumer],185,5,5563.560,2,2,5,225,Hibernating/At Risk
1,AA-10375,[Consumer],49,9,1056.390,4,5,2,452,Champions/Loyal Customers
2,AA-10480,[Consumer],260,4,1790.512,2,1,3,213,Hibernating/At Risk
3,AA-10645,[Consumer],234,6,5086.935,2,3,5,235,Can't Lose
4,AB-10015,[Consumer],446,3,886.156,1,1,1,111,Hibernating/At Risk
...,...,...,...,...,...,...,...,...,...,...
788,XP-21865,[Consumer],44,11,2374.658,4,5,3,453,Champions/Loyal Customers
789,YC-21895,[Corporate],5,5,5454.350,5,2,5,525,Promising/Potential Loyalists
790,YS-21880,[Corporate],10,8,6720.444,5,4,5,545,Champions/Loyal Customers
791,ZC-21910,[Consumer],55,13,8025.707,4,5,5,455,Champions/Loyal Customers


In [38]:
newColumns = ['Customer ID', 'RFM Score' ,'RFM Segment']
mergeThese = rfm[newColumns]

In [39]:
mergeThese

,Customer ID,RFM Score,RFM Segment
0,AA-10315,225,Hibernating/At Risk
1,AA-10375,452,Champions/Loyal Customers
2,AA-10480,213,Hibernating/At Risk
3,AA-10645,235,Can't Lose
4,AB-10015,111,Hibernating/At Risk
...,...,...,...
788,XP-21865,453,Champions/Loyal Customers
789,YC-21895,525,Promising/Potential Loyalists
790,YS-21880,545,Champions/Loyal Customers
791,ZC-21910,455,Champions/Loyal Customers


In [40]:
ConsumerCustomersRFM = rfm[rfm['Segment'] == 'Consumer']
CorporateCustomersRFM = rfm[rfm['Segment'] == 'Corporate']
HomeOfficeCustomerRFM = rfm[rfm['Segment'] == 'Home Office']

In [41]:
ConsumerCustomersRFM

,Customer ID,Segment,Recency,Frequency,Monetary,Recency Score,Frequency Score,Monetary Score,RFM Score,RFM Segment
0,AA-10315,[Consumer],185,5,5563.560,2,2,5,225,Hibernating/At Risk
1,AA-10375,[Consumer],49,9,1056.390,4,5,2,452,Champions/Loyal Customers
2,AA-10480,[Consumer],260,4,1790.512,2,1,3,213,Hibernating/At Risk
3,AA-10645,[Consumer],234,6,5086.935,2,3,5,235,Can't Lose
4,AB-10015,[Consumer],446,3,886.156,1,1,1,111,Hibernating/At Risk
...,...,...,...,...,...,...,...,...,...,...
784,VS-21820,[Consumer],222,4,1055.981,2,1,2,212,Hibernating/At Risk
787,WB-21850,[Consumer],50,11,6160.102,4,5,5,455,Champions/Loyal Customers
788,XP-21865,[Consumer],44,11,2374.658,4,5,3,453,Champions/Loyal Customers
791,ZC-21910,[Consumer],55,13,8025.707,4,5,5,455,Champions/Loyal Customers


In [42]:
lconsumerCustomers

,Customer ID,Segment,Sub-Category,Category,No_of_Orders,Total_Quantity,Order Date,Minimum Gaps,Maximum Gaps,Average Gaps,First Date of Purchase,Last Date of Purchase,Days Since Last Purchase,Flagged 1,Flagged 2
0,AA-10315,Consumer,Accessories,Technology,1,7,[2016-03-03 00:00:00],0.0,0.0,0.0,2014-03-31,2017-06-29,185,Yes,No
1,AA-10315,Consumer,Appliances,Office Supplies,2,5,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]",1186.0,1186.0,1186.0,2014-03-31,2017-06-29,185,Yes,Yes
2,AA-10315,Consumer,Binders,Office Supplies,2,4,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]",1186.0,1186.0,1186.0,2014-03-31,2017-06-29,185,Yes,Yes
3,AA-10315,Consumer,Fasteners,Office Supplies,1,1,[2016-03-03 00:00:00],0.0,0.0,0.0,2014-03-31,2017-06-29,185,Yes,No
4,AA-10315,Consumer,Furnishings,Furniture,1,2,[2014-09-15 00:00:00],0.0,0.0,0.0,2014-03-31,2017-06-29,185,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,ZD-21925,Consumer,Art,Office Supplies,1,2,[2014-08-27 00:00:00],0.0,0.0,0.0,2014-08-27,2017-11-06,55,Yes,No
5998,ZD-21925,Consumer,Binders,Office Supplies,1,4,[2014-08-27 00:00:00],0.0,0.0,0.0,2014-08-27,2017-11-06,55,Yes,No
5999,ZD-21925,Consumer,Furnishings,Furniture,2,9,"[2016-05-05 00:00:00, 2016-03-04 00:00:00]",62.0,62.0,62.0,2014-08-27,2017-11-06,55,No,No
6000,ZD-21925,Consumer,Paper,Office Supplies,1,2,[2016-08-07 00:00:00],0.0,0.0,0.0,2014-08-27,2017-11-06,55,Yes,No


In [43]:
ConsumerCustomersRFM = ConsumerCustomersRFM.drop(columns = {'Segment'})
CorporateCustomersRFM = CorporateCustomersRFM.drop(columns = {'Segment'})
HomeOfficeCustomerRFM = HomeOfficeCustomerRFM.drop(columns = {'Segment'})

In [44]:
lconsumerCustomers = pd.merge(lconsumerCustomers, ConsumerCustomersRFM, on = 'Customer ID', how = 'left')
lcorporateCustomers = pd.merge(lcorporateCustomers, CorporateCustomersRFM, on = 'Customer ID', how = 'left')
lhomeofficeCustomers = pd.merge(lhomeofficeCustomers, HomeOfficeCustomerRFM, on = 'Customer ID', how = 'left')

In [45]:
lconsumerCustomers

,Customer ID,Segment,Sub-Category,Category,No_of_Orders,Total_Quantity,Order Date,Minimum Gaps,Maximum Gaps,Average Gaps,...,Flagged 1,Flagged 2,Recency,Frequency,Monetary,Recency Score,Frequency Score,Monetary Score,RFM Score,RFM Segment
0,AA-10315,Consumer,Accessories,Technology,1,7,[2016-03-03 00:00:00],0.0,0.0,0.0,...,Yes,No,185,5,5563.560,2,2,5,225,Hibernating/At Risk
1,AA-10315,Consumer,Appliances,Office Supplies,2,5,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]",1186.0,1186.0,1186.0,...,Yes,Yes,185,5,5563.560,2,2,5,225,Hibernating/At Risk
2,AA-10315,Consumer,Binders,Office Supplies,2,4,"[2017-06-29 00:00:00, 2014-03-31 00:00:00]",1186.0,1186.0,1186.0,...,Yes,Yes,185,5,5563.560,2,2,5,225,Hibernating/At Risk
3,AA-10315,Consumer,Fasteners,Office Supplies,1,1,[2016-03-03 00:00:00],0.0,0.0,0.0,...,Yes,No,185,5,5563.560,2,2,5,225,Hibernating/At Risk
4,AA-10315,Consumer,Furnishings,Furniture,1,2,[2014-09-15 00:00:00],0.0,0.0,0.0,...,Yes,No,185,5,5563.560,2,2,5,225,Hibernating/At Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3119,ZD-21925,Consumer,Art,Office Supplies,1,2,[2014-08-27 00:00:00],0.0,0.0,0.0,...,Yes,No,55,5,1493.944,4,2,2,422,Promising/Potential Loyalists
3120,ZD-21925,Consumer,Binders,Office Supplies,1,4,[2014-08-27 00:00:00],0.0,0.0,0.0,...,Yes,No,55,5,1493.944,4,2,2,422,Promising/Potential Loyalists
3121,ZD-21925,Consumer,Furnishings,Furniture,2,9,"[2016-05-05 00:00:00, 2016-03-04 00:00:00]",62.0,62.0,62.0,...,No,No,55,5,1493.944,4,2,2,422,Promising/Potential Loyalists
3122,ZD-21925,Consumer,Paper,Office Supplies,1,2,[2016-08-07 00:00:00],0.0,0.0,0.0,...,Yes,No,55,5,1493.944,4,2,2,422,Promising/Potential Loyalists


In [46]:
lcorporateCustomers

,Customer ID,Segment,Sub-Category,Category,No_of_Orders,Total_Quantity,Order Date,Minimum Gaps,Maximum Gaps,Average Gaps,...,Flagged 1,Flagged 2,Recency,Frequency,Monetary,Recency Score,Frequency Score,Monetary Score,RFM Score,RFM Segment
0,AB-10600,Corporate,Accessories,Technology,1,6,[2017-10-11 00:00:00],0.0,0.0,0.000000,...,Yes,No,81,4,1515.862,3,1,2,312,Need Attention
1,AB-10600,Corporate,Binders,Office Supplies,2,19,"[2017-10-11 00:00:00, 2014-11-18 00:00:00]",1058.0,1058.0,1058.000000,...,Yes,Yes,81,4,1515.862,3,1,2,312,Need Attention
2,AB-10600,Corporate,Chairs,Furniture,1,4,[2017-10-11 00:00:00],0.0,0.0,0.000000,...,Yes,No,81,4,1515.862,3,1,2,312,Need Attention
3,AB-10600,Corporate,Furnishings,Furniture,1,2,[2015-11-30 00:00:00],0.0,0.0,0.000000,...,Yes,No,81,4,1515.862,3,1,2,312,Need Attention
4,AB-10600,Corporate,Labels,Office Supplies,1,3,[2015-02-18 00:00:00],0.0,0.0,0.000000,...,Yes,No,81,4,1515.862,3,1,2,312,Need Attention
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1799,YS-21880,Corporate,Chairs,Furniture,1,6,[2017-04-13 00:00:00],0.0,0.0,0.000000,...,Yes,No,10,8,6720.444,5,4,5,545,Champions/Loyal Customers
1800,YS-21880,Corporate,Envelopes,Office Supplies,1,9,[2015-07-25 00:00:00],0.0,0.0,0.000000,...,Yes,No,10,8,6720.444,5,4,5,545,Champions/Loyal Customers
1801,YS-21880,Corporate,Furnishings,Furniture,1,3,[2017-08-18 00:00:00],0.0,0.0,0.000000,...,Yes,No,10,8,6720.444,5,4,5,545,Champions/Loyal Customers
1802,YS-21880,Corporate,Storage,Office Supplies,4,19,"[2017-12-21 00:00:00, 2015-12-01 00:00:00, 201...",111.0,626.0,287.333333,...,Yes,Yes,10,8,6720.444,5,4,5,545,Champions/Loyal Customers


In [47]:
lhomeofficeCustomers

,Customer ID,Segment,Sub-Category,Category,No_of_Orders,Total_Quantity,Order Date,Minimum Gaps,Maximum Gaps,Average Gaps,...,Flagged 1,Flagged 2,Recency,Frequency,Monetary,Recency Score,Frequency Score,Monetary Score,RFM Score,RFM Segment
0,AB-10060,Home Office,Accessories,Technology,3,5,"[2016-01-09 00:00:00, 2017-07-05 00:00:00, 201...",24.0,519.0,271.5,...,Yes,Yes,106,8,7755.620,3,4,5,345,Frequent but not Recent
1,AB-10060,Home Office,Appliances,Office Supplies,1,2,[2016-03-13 00:00:00],0.0,0.0,0.0,...,Yes,No,106,8,7755.620,3,4,5,345,Frequent but not Recent
2,AB-10060,Home Office,Binders,Office Supplies,2,9,"[2016-01-09 00:00:00, 2016-01-09 00:00:00, 201...",64.0,64.0,64.0,...,No,No,106,8,7755.620,3,4,5,345,Frequent but not Recent
3,AB-10060,Home Office,Chairs,Furniture,1,2,[2017-06-11 00:00:00],0.0,0.0,0.0,...,Yes,No,106,8,7755.620,3,4,5,345,Frequent but not Recent
4,AB-10060,Home Office,Furnishings,Furniture,1,2,[2016-03-13 00:00:00],0.0,0.0,0.0,...,Yes,No,106,8,7755.620,3,4,5,345,Frequent but not Recent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,VT-21700,Home Office,Bookcases,Furniture,1,5,[2015-05-04 00:00:00],0.0,0.0,0.0,...,No,No,972,2,1736.596,1,1,2,112,Hibernating/At Risk
1070,VT-21700,Home Office,Furnishings,Furniture,1,5,"[2015-05-04 00:00:00, 2015-05-04 00:00:00]",0.0,0.0,0.0,...,No,No,972,2,1736.596,1,1,2,112,Hibernating/At Risk
1071,VT-21700,Home Office,Labels,Office Supplies,1,10,[2014-06-06 00:00:00],0.0,0.0,0.0,...,No,No,972,2,1736.596,1,1,2,112,Hibernating/At Risk
1072,VT-21700,Home Office,Phones,Technology,1,7,"[2015-05-04 00:00:00, 2015-05-04 00:00:00]",0.0,0.0,0.0,...,No,No,972,2,1736.596,1,1,2,112,Hibernating/At Risk


In [48]:
lconsumerCustomers.to_csv('Consumer Customers with RFM.csv')
lcorporateCustomers.to_csv('Coporate Customers with RFM.csv')
lhomeofficeCustomers.to_csv('Home Office Customers with RFM.csv')

## Recency:

Measures how recently a customer made a purchase. 
Here, a higher recency value indicates that the customer has made a purchase recently.
Here, on a scale of 1-5, 5 being the lowest value, indicates that the customer has recently made a purchase. Whereas, 1 being the highest value, indicates that its been a very long time since the customer has made a purchase. 

## Frequency

Measures how often a customer makes purchases within a specific time frame.
Here, a higher frequency value indicates that the customer purchases frequently. 
Here, on a scale of 1-5, 5 being the highest value, indicates that the customer purchases frequently. Whereas, 1 being the lowest value, indicates that the customer barely makes any purchases.

## Monetary

Measures how much money a customer spends over a specific period. 
Here, a higher monetary value indicates that the customer has spent alot. Customers with higher monetary values contribute significantly to the company's revenue. 
Here, on a scale of 1-5, 5 being the highest value, indicates that the customer contributes significantly to the company's revenue. Whereas, 1 being the lowest value, indicates that the customer's contribution to the company's revenue is the least significant. 

In [49]:
frequencyDistributionRFM = rfm['RFM Segment'].value_counts()
frequencyDistributionRFM = frequencyDistributionRFM.reset_index()

In [50]:
frequencyDistributionRFM

,RFM Segment,count
0,Promising/Potential Loyalists,194
1,Hibernating/At Risk,184
2,Can't Lose,132
3,Champions/Loyal Customers,125
4,Need Attention,103
5,Frequent but not Recent,55


In [51]:
newOrder = [
    'Champions/Loyal Customers',
    'Promising/Potential Loyalists',
    "Can't Lose",
    'Frequent but not Recent',
    'Need Attention',
    'Hibernating/At Risk'
]

In [52]:
frequencyDistributionRFM = frequencyDistributionRFM.set_index('RFM Segment').reindex(newOrder).reset_index()
frequencyDistributionRFM

,RFM Segment,count
0,Champions/Loyal Customers,125
1,Promising/Potential Loyalists,194
2,Can't Lose,132
3,Frequent but not Recent,55
4,Need Attention,103
5,Hibernating/At Risk,184


## Hibernating/At Risk(170):

These are the customers who have not made purchases recently, and are not frequent. It has been a very long time that they have visited and have made very few visits the store.

## Promising/Potential Loyalists(161):

These are the customers who have made purchases recently but are either not frequent or are moderately frequent. They have visited the store several time recently, would pose as potential loyal customers. 

## Champions/Loyal Customers(158):

These are the customers who make frequent purchases. They contribute significantly to the company's revenue. 

## Can't Lose(146):

These are the customers who used to visit frequently but then, it's been a long time that they have visited the store. The company should focus on trying to grab these customer's attention. 

## Need Attention(91):

These are the customers who have visited quite recently, but they are moderately frequent. 

## Frequent but not Recent(67):

These are the customers who used to visit the store frequently. 